In [5]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.optimize import minimize
from pypfopt import expected_returns, risk_models, EfficientFrontier

In [6]:
tickers = ('AAPL', 'NVDA')

In [7]:
conn = sqlite3.connect('sp500_stocks.db')
cursor = conn.cursor()
cursor.execute(f"SELECT ticker, date, close FROM price WHERE ticker IN {tuple(tickers)} AND date BETWEEN '2025-01-01' AND '2025-08-01'")
data = cursor.fetchall()
conn.close()
df = pd.DataFrame(data, columns=['ticker', 'date', 'close'])
df = df.pivot(index='date', columns='ticker', values='close').dropna()
df

ticker,AAPL,NVDA
date,,
2025-01-02 00:00:00,243.263199,138.287674
2025-01-03 00:00:00,242.774368,144.446686
2025-01-06 00:00:00,244.410416,149.405869
2025-01-07 00:00:00,241.627136,140.117371
2025-01-08 00:00:00,242.115936,140.087387
...,...,...
2025-07-25 00:00:00,213.880005,173.500000
2025-07-28 00:00:00,214.050003,176.750000
2025-07-29 00:00:00,211.270004,175.509995


In [8]:
returns = df.pct_change().dropna()
returns

ticker,AAPL,NVDA
date,,
2025-01-03 00:00:00,-0.002009,0.044538
2025-01-06 00:00:00,0.006739,0.034332
2025-01-07 00:00:00,-0.011388,-0.062170
2025-01-08 00:00:00,0.002023,-0.000214
2025-01-10 00:00:00,-0.024104,-0.029976
...,...,...
2025-07-25 00:00:00,0.000561,-0.001381
2025-07-28 00:00:00,0.000795,0.018732
2025-07-29 00:00:00,-0.012988,-0.007016


In [9]:
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

In [14]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe(risk_free_rate=0.04)
cleaned_weights = ef.clean_weights()
print("Optimal Weights:", weights)

Optimal Weights: OrderedDict([('AAPL', 0.0), ('NVDA', 1.0)])


In [18]:
for key in cleaned_weights.keys():
    print(cleaned_weights[key])

0.0
1.0


In [20]:
df['Value'] = 0
for ticker in weights.keys():
    df['Value'] += (1 + df.loc[:,ticker].pct_change()).cumprod() * (1000 * weights[ticker])
df['Value']

date
2025-01-02 00:00:00            NaN
2025-01-03 00:00:00    1044.537678
2025-01-06 00:00:00    1080.399028
2025-01-07 00:00:00    1013.231090
2025-01-08 00:00:00    1013.014270
                          ...     
2025-07-25 00:00:00    1254.630981
2025-07-28 00:00:00    1278.132714
2025-07-29 00:00:00    1269.165859
2025-07-30 00:00:00    1296.355627
2025-07-31 00:00:00    1286.231737
Name: Value, Length: 144, dtype: float64

In [11]:
ef.portfolio_performance(verbose=True, risk_free_rate=0.05)

Expected annual return: 55.8%
Annual volatility: 58.8%
Sharpe Ratio: 0.86


(0.5582834571000714, 0.5883104509432074, 0.8639714903673172)